In [1]:
#!pip install sklearn
#!pip install xgboost
#!pip install keras

In [2]:
import ISee
import Profiler

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer

import numpy as np

import Code_vin
import NullHypothesis

import re #REGEX sera utilizado para implementar as heuristicas de pre-processamento
import random

Using TensorFlow backend.


In [3]:
def multisub(patterns, strns, text_pos):
    for strn in strns:
        for pattern, subst in patterns:
            strn[text_pos] = re.sub(pattern, subst, strn[text_pos])
    return strns

In [4]:
#A: bigrams
#B: all characters to lowercase
#C: accentuation removal
#D: special character treatment
#E: stop-words removal
#F: twitter user names removal
#G: twitter topics removal
#H: Reduction of laugh expressions to a common token
laugh_token = ' _laugh_ '
heuristics = {
    'A': [('not( [^ ]+)', 'not_\1')],
    'B': [('([A-Z]*)', lambda x: x.group(0).lower())],
    'C': [('á', 'a'), ('é', 'e'), ('í', 'i'), ('ó', 'o'), ('ú', 'u'), ('à', 'a'), ('ã', 'a'), ('ẽ', 'e'), ('ĩ', 'i'), ('õ', 'o'), ('ũ', 'u'),  ('â', 'a'), ('ê', 'e'), ('ñ', 'n')],
    'D': [(':\)', ''), (':\(', ''), (':\^\)', ''), (':p', ''), (':3', ''), (':c', ''), ('c:', ''), (':o', '')],
    'E': [(' the ', ' '), (' a ', ' '), (' e ', ' '), (' o ', ' ')],
    'F': [('@[^ ]+', ''), ('@[^ ]+ ', '')],
    'G': [('#[^ ]+', ''), ('#[^ ]+ ', '')],
    'H': [(' [ashu]{3,} ', laugh_token), (' [ah]{3,} ', laugh_token), (' [eh]{3,} ', laugh_token), (' k{4,} ', laugh_token), (' [rs]{3,} ', laugh_token), (' [ehu]{3,} ', laugh_token)],
}

In [5]:
#Configuracoes
heuristics_in_use = 'ABCDEFGH' #ABCDEFGH

max_features = 1000
embedding_dims = 50
maxlen = 50

kfold = 5

will_shuffle = True
shuffle_seed = 1234

n_est = 1000

In [6]:
prf = Profiler.Milestones()

[Profiler starting...]


In [7]:
print("Creating ensemble")
ensemble = ISee.Ensemble()
prf.add_milestone("Created ensemble object")

Creating ensemble


[Profiler] @ 22:28:58: Created ensemble object


In [8]:
print("Setting encoder & tokenizer")
ensemble.set_encoder(LabelEncoder())
ensemble.set_tokenizer(Tokenizer(max_features))
prf.add_milestone("Set encoder & tokenizer")

Setting encoder & tokenizer


[Profiler] @ 22:28:58: Set encoder & tokenizer


In [9]:
print("Reading file")
file_alias = 'portugues' #'portugues'

if file_alias == 'brexit':
    text_position = 4
    class_position = 5
    file_name = "brexit_blog_corpus.csv"
if file_alias == 'portugues':
    text_position = 0
    class_position = 1
    file_name = "tweets_br.csv"
    
with open("datasets/" + file_name) as f:
    lines = [line.rstrip('\n').split('\t') for line in f][1:]
    print("Read {} lines".format(len(lines)))
    print("Applying heuristics " + heuristics_in_use + " (" + str(len(heuristics_in_use)) + ")")
    for h in heuristics_in_use:
        lines = multisub(heuristics[h], lines, text_position)
    line_no = 0
    for line in lines:
        line.append(line_no)
        line_no += 1
prf.add_milestone("Read file")

if will_shuffle:
    print("Shuffling lines with seed {}".format(shuffle_seed))
    random.seed(shuffle_seed)
    random.shuffle(lines)

#print(str(lines[15]))
textos  = [str(a[text_position])  for a in lines]
classes = [str(b[class_position]) for b in lines]
n_classes = len(set(classes))
indexes = [c[-1] for c in lines]
prf.add_milestone("Aggregated data")

Reading file
Read 6360 lines
Applying heuristics ABCDEFGH (8)
Shuffling lines with seed 1234


[Profiler] @ 22:28:59: Read file
[Profiler] @ 22:28:59: Aggregated data


In [10]:
print("Setting data")
ensemble.set_datasets(textos, classes, indexes)
prf.add_milestone("Set datasets")

Setting data


[Profiler] @ 22:28:59: Set datasets


In [11]:
print("Fitting encoder & tokenizer")
ensemble.fit_encoder()
ensemble.fit_tokenizer()
prf.add_milestone("Fitted encoder & tokenizer")

Fitting encoder & tokenizer


[Profiler] @ 22:29:00: Fitted encoder & tokenizer


In [12]:
print("Splitting data")
ensemble.split_data(shuffle_seed=1337, purge_duplicates=False)
prf.add_milestone("Split data")

Splitting data


[Profiler] @ 22:29:00: Split data


In [13]:
print("Adding models")
##ensemble.add('Random', NullHypothesis.RandomClassifier())
#ensemble.add('XGB',  XGBClassifier(n_estimators=n_est, learning_rate=0.01, max_depth=6, subsample=0.65, colsample_bytree=0.25, gamma=5))
#ensemble.add('Forest', RandomForestClassifier(n_estimators=90))
#ensemble.add('Naive-B',  GaussianNB())
#ensemble.add('SVM_T', svm.SVC(gamma='scale'))
ensemble.add('RNC_binary_crossentropy_1', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=maxlen, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='binary_crossentropy'), uses_one_hot=False, uses_categorical=True, uses_argmax=True)
ensemble.add('RNC_binary_crossentropy_2', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=maxlen, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='binary_crossentropy'), uses_one_hot=False, uses_categorical=True, uses_argmax=True)
ensemble.add('RNC_binary_crossentropy_3', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=maxlen, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='binary_crossentropy'), uses_one_hot=False, uses_categorical=True, uses_argmax=True)
#ensemble.add('RNC_1T_CF', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=max_features, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='binary_crossentropy'), uses_one_hot=True, uses_categorical=True, uses_argmax=True)
ensemble.add('RNC_categorical_crossentropy_1', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=maxlen, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='categorical_crossentropy'), uses_one_hot=False, uses_categorical=True, uses_argmax=True)
ensemble.add('RNC_categorical_crossentropy_2', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=maxlen, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='categorical_crossentropy'), uses_one_hot=False, uses_categorical=True, uses_argmax=True)
ensemble.add('RNC_categorical_crossentropy_3', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=maxlen, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='categorical_crossentropy'), uses_one_hot=False, uses_categorical=True, uses_argmax=True)
#ensemble.add('RNC_1T_CT', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=max_features, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='categorical_crossentropy'), uses_one_hot=True, uses_categorical=True, uses_argmax=True)
prf.add_milestone("Added models")

Adding models


[Profiler] @ 22:29:01: Added models


In [14]:
print("Training ensemble")
ensemble.train(max_features, maxlen, profiler=prf)
prf.add_milestone("Trained ensemble")

[Profiler] @ 22:29:01: Starting training


Training ensemble
Training with the 1-st set


[Profiler] @ 22:29:02: Fold 1 - Training model RNC_binary_crossentropy_1
[Profiler] @ 22:29:42: Fold 1 - Training model RNC_binary_crossentropy_2
[Profiler] @ 22:30:20: Fold 1 - Training model RNC_binary_crossentropy_3
[Profiler] @ 22:30:59: Fold 1 - Training model RNC_categorical_crossentropy_1
[Profiler] @ 22:31:37: Fold 1 - Training model RNC_categorical_crossentropy_2
[Profiler] @ 22:32:16: Fold 1 - Training model RNC_categorical_crossentropy_3


Training with the 2-nd set


[Profiler] @ 22:32:57: Fold 2 - Training model RNC_binary_crossentropy_1
[Profiler] @ 22:33:34: Fold 2 - Training model RNC_binary_crossentropy_2
[Profiler] @ 22:34:13: Fold 2 - Training model RNC_binary_crossentropy_3
[Profiler] @ 22:34:50: Fold 2 - Training model RNC_categorical_crossentropy_1
[Profiler] @ 22:35:28: Fold 2 - Training model RNC_categorical_crossentropy_2
[Profiler] @ 22:36:07: Fold 2 - Training model RNC_categorical_crossentropy_3


...

[Profiler] @ 22:40:36: Fold 4 - Training model RNC_binary_crossentropy_1
[Profiler] @ 22:41:15: Fold 4 - Training model RNC_binary_crossentropy_2
[Profiler] @ 22:41:53: Fold 4 - Training model RNC_binary_crossentropy_3
[Profiler] @ 22:42:31: Fold 4 - Training model RNC_categorical_crossentropy_1
[Profiler] @ 22:43:10: Fold 4 - Training model RNC_categorical_crossentropy_2
[Profiler] @ 22:43:48: Fold 4 - Training model RNC_categorical_crossentropy_3


Training with the 5-th set


[Profiler] @ 22:44:27: Fold 5 - Training model RNC_binary_crossentropy_1
[Profiler] @ 22:45:05: Fold 5 - Training model RNC_binary_crossentropy_2
[Profiler] @ 22:45:44: Fold 5 - Training model RNC_binary_crossentropy_3
[Profiler] @ 22:46:22: Fold 5 - Training model RNC_categorical_crossentropy_1
[Profiler] @ 22:47:00: Fold 5 - Training model RNC_categorical_crossentropy_2
[Profiler] @ 22:47:39: Fold 5 - Training model RNC_categorical_crossentropy_3


Finished training


[Profiler] @ 22:48:17: Trained ensemble


In [15]:
with open('eval_results_' + file_name[:-4] + '.txt', 'a') as f:
    f.write(ensemble.evalHistories())

1-st fold
	Model RNC_binary_crossentropy_1
		Max  acc: 95.19%
		Mean acc: 87.41%
		Max  val_acc: 82.23%
		Mean val_acc: 81.43%
		Max  loss: 57.27%
		Mean loss: 28.96%
		Max  val_loss: 52.20%
		Mean val_loss: 43.33%
	Model RNC_binary_crossentropy_2
		Max  acc: 94.47%
		Mean acc: 86.97%
		Max  val_acc: 82.74%
		Mean val_acc: 81.50%
		Max  loss: 57.68%
		Mean loss: 29.84%
		Max  val_loss: 51.70%
		Mean val_loss: 42.62%
	Model RNC_binary_crossentropy_3
		Max  acc: 94.73%
		Mean acc: 86.84%
		Max  val_acc: 83.77%
		Mean val_acc: 82.05%
		Max  loss: 58.58%
		Mean loss: 29.68%
		Max  val_loss: 60.76%
		Mean val_loss: 43.79%
	Model RNC_categorical_crossentropy_1
		Max  acc: 95.26%
		Mean acc: 87.12%
		Max  val_acc: 82.55%
		Mean val_acc: 81.60%
		Max  loss: 58.03%
		Mean loss: 29.49%
		Max  val_loss: 57.70%
		Mean val_loss: 43.83%
	Model RNC_categorical_crossentropy_2
		Max  acc: 94.69%
		Mean acc: 87.43%
		Max  val_acc: 82.70%
		Mean val_acc: 81.53%
		Max  loss: 55.84%
		Mean loss: 29.04%
		M

In [16]:
print("Testing ensemble")
res = ensemble.test(max_features, maxlen, profiler=prf)
prf.add_milestone("Tested ensemble")

Testing ensemble
Testing with the 1-st set


[Profiler] @ 22:48:17: Starting testing
[Profiler] @ 22:48:17: Fold 1 - Testing model RNC_binary_crossentropy_1
[Profiler] @ 22:48:18: Fold 1 - Tested model RNC_binary_crossentropy_1
[Profiler] @ 22:48:18: Fold 1 - Testing model RNC_binary_crossentropy_2
[Profiler] @ 22:48:18: Fold 1 - Tested model RNC_binary_crossentropy_2
[Profiler] @ 22:48:18: Fold 1 - Testing model RNC_binary_crossentropy_3
[Profiler] @ 22:48:18: Fold 1 - Tested model RNC_binary_crossentropy_3
[Profiler] @ 22:48:18: Fold 1 - Testing model RNC_categorical_crossentropy_1
[Profiler] @ 22:48:19: Fold 1 - Tested model RNC_categorical_crossentropy_1
[Profiler] @ 22:48:19: Fold 1 - Testing model RNC_categorical_crossentropy_2
[Profiler] @ 22:48:19: Fold 1 - Tested model RNC_categorical_crossentropy_2
[Profiler] @ 22:48:19: Fold 1 - Testing model RNC_categorical_crossentropy_3
[Profiler] @ 22:48:19: Fold 1 - Tested model RNC_categorical_crossentropy_3
[Profiler] @ 22:48:20: Fold 2 - Testing model RNC_binary_crossentropy_1


Testing with the 2-nd set


[Profiler] @ 22:48:20: Fold 2 - Tested model RNC_binary_crossentropy_1
[Profiler] @ 22:48:20: Fold 2 - Testing model RNC_binary_crossentropy_2
[Profiler] @ 22:48:20: Fold 2 - Tested model RNC_binary_crossentropy_2
[Profiler] @ 22:48:20: Fold 2 - Testing model RNC_binary_crossentropy_3
[Profiler] @ 22:48:20: Fold 2 - Tested model RNC_binary_crossentropy_3
[Profiler] @ 22:48:20: Fold 2 - Testing model RNC_categorical_crossentropy_1
[Profiler] @ 22:48:21: Fold 2 - Tested model RNC_categorical_crossentropy_1
[Profiler] @ 22:48:21: Fold 2 - Testing model RNC_categorical_crossentropy_2
[Profiler] @ 22:48:21: Fold 2 - Tested model RNC_categorical_crossentropy_2
[Profiler] @ 22:48:21: Fold 2 - Testing model RNC_categorical_crossentropy_3
[Profiler] @ 22:48:21: Fold 2 - Tested model RNC_categorical_crossentropy_3


Testing with the 3-rd set


...

[Profiler] @ 22:48:22: Fold 4 - Testing model RNC_binary_crossentropy_1
[Profiler] @ 22:48:23: Fold 4 - Tested model RNC_binary_crossentropy_1
[Profiler] @ 22:48:23: Fold 4 - Testing model RNC_binary_crossentropy_2
[Profiler] @ 22:48:23: Fold 4 - Tested model RNC_binary_crossentropy_2
[Profiler] @ 22:48:23: Fold 4 - Testing model RNC_binary_crossentropy_3
[Profiler] @ 22:48:23: Fold 4 - Tested model RNC_binary_crossentropy_3
[Profiler] @ 22:48:23: Fold 4 - Testing model RNC_categorical_crossentropy_1
[Profiler] @ 22:48:23: Fold 4 - Tested model RNC_categorical_crossentropy_1
[Profiler] @ 22:48:23: Fold 4 - Testing model RNC_categorical_crossentropy_2
[Profiler] @ 22:48:23: Fold 4 - Tested model RNC_categorical_crossentropy_2
[Profiler] @ 22:48:23: Fold 4 - Testing model RNC_categorical_crossentropy_3
[Profiler] @ 22:48:24: Fold 4 - Tested model RNC_categorical_crossentropy_3


Testing with the 5-th set


[Profiler] @ 22:48:24: Fold 5 - Testing model RNC_binary_crossentropy_1
[Profiler] @ 22:48:24: Fold 5 - Tested model RNC_binary_crossentropy_1
[Profiler] @ 22:48:24: Fold 5 - Testing model RNC_binary_crossentropy_2
[Profiler] @ 22:48:24: Fold 5 - Tested model RNC_binary_crossentropy_2
[Profiler] @ 22:48:24: Fold 5 - Testing model RNC_binary_crossentropy_3
[Profiler] @ 22:48:24: Fold 5 - Tested model RNC_binary_crossentropy_3
[Profiler] @ 22:48:24: Fold 5 - Testing model RNC_categorical_crossentropy_1
[Profiler] @ 22:48:24: Fold 5 - Tested model RNC_categorical_crossentropy_1
[Profiler] @ 22:48:24: Fold 5 - Testing model RNC_categorical_crossentropy_2
[Profiler] @ 22:48:25: Fold 5 - Tested model RNC_categorical_crossentropy_2
[Profiler] @ 22:48:25: Fold 5 - Testing model RNC_categorical_crossentropy_3


Finished testing


[Profiler] @ 22:48:25: Fold 5 - Tested model RNC_categorical_crossentropy_3
[Profiler] @ 22:48:25: Tested ensemble


In [17]:
ensemble.evaluate(verbose=True, file_d=open('manual_results_' + file_name[:-4] + '.txt', 'a'))
prf.add_milestone("Finished evaluating")

Evaluating 1-th fold
	     -  Correct - RNC_binary_crossentropy_1 - RNC_binary_crossentropy_2 - RNC_binary_crossentropy_3 - RNC_categorical_crossentropy_1 - RNC_categorical_crossentropy_2 - RNC_categorical_crossentropy_3 -   Major.- Phrase
	   0 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* tem gente que tem dinheiro tem gente que tem um ps4 eu tenho sono
	   1 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* o filme de warcraft vai ser foda de god of war entao porra ia ser epico
	   2 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* mas melhor franquia da nintendo pra mim legend of zelda, mano mt foda *----*
	   3 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0*  manda um abraço pra itaporanga-se acompanhando esse mega show de basquete. s

	 116 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* """ michel temer em vitoria: "" sociedade quer que pmdb seja governo do pais"" deus nos livre 👀"
	 117 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0*  gatonaaa, meus parabens nega , tudo de bom hoje sempre , muito sucesso em _laugh_ vida ! beijao 😙👏🎉
	 118 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* eu: posso ter um pouco de privacidade, por favor?  mae: claro! arranje _laugh_ propria casa.
	 119 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0*  fantastico. 😂
	 120 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* gosto assim.
	 121 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -      

	 244 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* eu odeio toda gente que fez don't judge challenge
	 245 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* tava assistindo quarteto fantastico ai bateu aquele sono fui dormir so que eu nao tava com sono filme que muito chato
	 246 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* ir ao estadio ver jogo de futebol emocionante, mas assistir um jogo de basquete da nba de perto sensacional
	 247 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* 14- ed sheeran eh um artista maravilhoso, ele compoe musicas lindas, tem uma voz maravilhosa lutou mto pra chegar onde chegou, ele sim eh+
	 248 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* 

	 381 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* eu so queria 10 caixas de montebello da cacau show n eh pedir demais jesus
	 382 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0*  marcelo sabe por que vôlei legal de assistir? pq nao tenho time preferido todos sao equipes muito fortes, legal de ver.
	 383 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0*    ele incrivelmente fantastico
	 384 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* eu gosto tanto dessa header
	 385 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* odeio quando entro em algum video sem querer musica para
	 386 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0*

	 508 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* bom dia!!! desejo uma quarta-feira iluminada repleta de resultados positivos todos.
	 509 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* acho q vo compra primal ta bem legal ele 
	 510 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* nada de star wars nem star trek queria star com vc
	 511 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* to cansada ja de sao paulo
	 512 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* estou lendo um livro maravilhoso
	 513 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0*  poxa aline, assim nao ne filha, ninguem esta disputado fe. e

...

	 714 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* este resultado do derbi madrileno esta mesmo bom.
	 715 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* "koeman foi um ""monstro"" como jogador...espetacular zagueiro,eximio batedor de falta um lider em campo....fantÁstico ! 
	 716 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* """ta com voz de sono foi mal se te acordei""  foi mal mesmo foi pessimo horrivel lamentavel deploravel lastimavel ridiculo repugnante odiei"
	 717 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0*  um mais maravilhoso que outro
	 718 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0*  nunca tive, minha professora super legal
	 719 -        1.0 -        

	 844 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* fone da apple sagrado
	 845 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0*  seu dente falso seu cabelo horroroso
	 846 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0*   manda beijo pra mim daiana de blumenau santa catarina adoro vcs todos...
	 847 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* tendo uma morte terrivel com preço do material
	 848 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* minha cidade assim: cai vivo, cai tim, cai wifi, cai força de vontade, cai vida, vai tudo de uma vez.
	 849 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* Ô vou ate apag

	 984 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* odeio toda apple _laugh_ malditas tecnologias. nao da pra brincar desse jeito, 
	 985 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* meu aleatorio começou tocar thought of you quase chorei de emoçao, fav do believe
	 986 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* o atendente da apple tem mais paciencia cmg que meus ex
	 987 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* so eles mesmo pra melhorar totalmente meu dia, me lembrar q tem mt gente pior q eu
	 988 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* pior de td eh q agr nao vai ter mais futebol
	 989 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       

	1134 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* celular roubado na merda daquela escola
	1135 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* mcgregor pica demais
	1136 -        1.0 -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0* -        1.0*  maravilhosa demais msm
	1137 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* what kind of apple turns black  nao sei miga quem sabe uma envenenada
	1138 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* nunca houve na historia desse pais, um governo, um congresso, um stf tao mediocres, do ponto de vista tecnico moral.
	1139 -       -1.0 -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* -       -1.0* corretor da _laugh_ me imp

[Profiler] @ 22:48:34: Finished evaluating


In [18]:
print("Profiler data:")
prf.exhibit(max_d=10)

Profiler data:
0=1=2=3=4=5=6=7=8=9=10==========11=========12=========13=========14=========15=========16=========17=========18=========19=========20=========21=========22=========23=========24=========25=========26=========27=========28=========29=========30=========31=========32=========33=========34=========35=========36=========37=========38=========39=========40=41=42=43=44=45=46=47=48=49=50=51=52=53=54=55=56=57=58=59=60=61=62=63=64=65=66=67=68=69=70=71=72=73=74=75=76=77=78=79=80=81=82=83=84=85=86=87=88=89=90=91=92=93=94=95=96=97=98=99=100=101=102===103
  0: Start - 20/05/2019 - 22:28:58
  1: Created ensemble object - 20/05/2019 - 22:28:58 - 0.12s
  2: Set encoder & tokenizer - 20/05/2019 - 22:28:58 - 0.12s
  3: Read file - 20/05/2019 - 22:28:59 - 0.88s
  4: Aggregated data - 20/05/2019 - 22:28:59 - 0.01s
  5: Set datasets - 20/05/2019 - 22:28:59 - 0.01s
  6: Fitted encoder & tokenizer - 20/05/2019 - 22:29:00 - 0.51s
  7: Split data - 20/05/2019 - 22:29:00 - 0.40s
  8: Added models